In [44]:
import os, time,json
import tmdbsimple as tmdb 
import pandas as pd 
from tqdm.notebook import tqdm_notebook

In [45]:
#Load and the login credentials from your new JSON file in your notebook.
import json
with open('/Users/Testing/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key', 'client-id'])

In [46]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

### Designate a folder

In [52]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['final_tmdb_data_2000.csv',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

### Define Your Functions

In [53]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [54]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    # save the .info .release dictionaries
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
            info['certification'] = c['certification']
    
    return info

In [55]:
def read_and_fix_json(JSON_FILE):
    """Attempts to read in json file of records and fixes the final character
    to end with a ] if it errors.
    
    Args:
        JSON_FILE (str): filepath of JSON file
        
    Returns:
        DataFrame: the corrected data from the bad json file
    """
    try: 
        previous_df =  pd.read_json(JSON_FILE)
    
    ## If read_json throws an error
    except:
        
        ## manually open the json file
        with open(JSON_FILE,'r+') as f:
            ## Read in the file as a STRING
            bad_json = f.read()
            
            ## if the final character doesn't match first, select the right bracket
            first_char = bad_json[0]
            final_brackets = {'[':']', 
                           "{":"}"}
            ## Select expected final brakcet
            final_char = final_brackets[first_char]
            
            ## if the last character in file doen't match the first char, add it
            if bad_json[-1] != final_char:
                good_json = bad_json[:-1]
                good_json+=final_char
            else:
                raise Exception('ERROR is not due to mismatched final bracket.')
            
            ## Rewind to start of file and write new good_json to disk
            f.seek(0)
            f.write(good_json)
           
        ## Load the json file again now that its fixed
        previous_df =  pd.read_json(JSON_FILE)
        
    return previous_df

In [58]:
def create_json(JSON_FILE):
    """create json file"""
    with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

### Load in the Title Basics data

In [56]:
# Load inthe dataframe from project part 1 as basics:
basics = pd.read_csv('title_basics_clean.csv')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
2,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama
3,tt0102362,movie,Istota,Istota,0,2000,NaN,80,"Drama,Romance"
4,tt0112912,movie,Dune 7,Dune 7,0,2002,NaN,97,Adventure


Define a list of the Years to Extract from the API

In [57]:
# Define a list of the Years to Extract from the API
YEARS_TO_GET = [2000,2001]

# Define an errors list
errors = [ ]

### Start OUTER loop

In [60]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists (file doesnt exist at first, just folders)
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        create_json(JSON_FILE)
    else:
        print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
        ## delete file and confirm it no longer exits and create another
        os.remove(JSON_FILE)
        create_json(JSON_FILE)
    #Saving new year as the current df (we are to break up title basic by year)
    df = basics.loc[basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    # Load existing data from json into a dataframe called "previous_df" (safegueard)
    previous_df = read_and_fix_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    movie_ids_to_get

    # Start INNER Loop
    #Get index and movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id) 
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])


    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

[!] Data/tmdb_api_results_2000.json already exists. Deleting previous file...


Movies from 2000:   0%|          | 0/2739 [00:00<?, ?it/s]

[!] Data/tmdb_api_results_2001.json already exists. Deleting previous file...


Movies from 2001:   0%|          | 0/2868 [00:00<?, ?it/s]

In [61]:
print(f"- Total errors: {len(errors)}")

- Total errors: 1098
